In [1]:
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,original_publication_date,corpus_edition_date,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
218460,Anti-Oedipus,Deleuze,continental,But when the dualism passed into a death insti...,But when the dualism passed into a death insti...,1972,1997,139,but when the dualism passed into a death insti...,"['but', 'when', 'the', 'dualism', 'passed', 'i...",but when the dualism pass into a death instin...
365890,On Benefits,Seneca,stoicism,When compose what we may sell to scholars?,When compose what we may sell to scholars?,59,2017,42,when compose what we may sell to scholars?,"['when', 'compose', 'what', 'we', 'may', 'sell...",when compose what -PRON- may sell to scholar ?
252841,Critique Of Judgement,Kant,german_idealism,Thus they indicate that teleology must look to...,Thus they indicate that teleology must look to...,1790,2007,97,thus they indicate that teleology must look to...,"['thus', 'they', 'indicate', 'that', 'teleolog...",thus -PRON- indicate that teleology must look...
210859,Difference And Repetition,Deleuze,continental,The shortest is not simply the schema of the c...,The shortest is not simply the schema of the c...,1968,1994,212,the shortest is not simply the schema of the c...,"['the', 'shortest', 'is', 'not', 'simply', 'th...",the short be not simply the schema of the con...
62218,Aristotle - Complete Works,Aristotle,aristotle,"For here, for reasons already given, there are...","For here, for reasons already given, there are...",-320,1991,146,"for here, for reasons already given, there are...","['for', 'here', 'for', 'reasons', 'already', '...","for here , for reason already give , there be..."


In [3]:
len(df)

376865

In [4]:
for_db = df
for_db['date'] = for_db['original_publication_date']
for_db['date'] = for_db['date'].apply(lambda x: str(x)[1:]+' BC' if x < 0 else str(x))
for_db['sentence'] = for_db['sentence_str']
for_db['school'] = for_db['school'].apply(lambda x: x.replace('_', ' ').title())
for_db = for_db.drop(['sentence_spacy', 
                      'sentence_length',
                      'sentence_lowered', 
                      'sentence_str', 
                      'tokenized_txt', 
                      'lemmatized_str',
                      'corpus_edition_date',
                      'original_publication_date'], axis=1)
for_db.columns = [i.upper() for i in for_db.columns]

for_db.sample(5)


,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
51578,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,This will certainly be the body that naturally...
337325,The Second Sex,Beauvoir,Feminism,1949,Or unconscious gratitude for a good digestion ...
325541,"A General Theory Of Employment, Interest, And ...",Keynes,Capitalism,1936,Another school of thought finds the solution o...
263851,The System Of Ethics,Fichte,German Idealism,1798,Not only does the intellect engage from now on...
363828,De Veritate,Anselm,Scholasticism,1086,While virtually all the Southern white men wer...


In [5]:
for_db.sample(5)

,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
59866,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,"If stags be mutilated when, by reason of their..."
336805,Vindication Of The Rights Of Woman,Wollstonecraft,Feminism,1792,"Ten years have elapsed, and no one has yet fel..."
322099,On The Principles Of Political Economy And Tax...,Ricardo,Capitalism,1817,Neither would any greater proportion of the gr...
248406,Critique Of Practical Reason,Kant,German Idealism,1788,Or suppose that someone recornrnend ed to as a...
154215,Lewis - Papers,Lewis,Analytic,1985,Many propositions must have infinitesimal Cval...


In [6]:
for_db['SCHOOL'].value_counts(normalize=True)

Analytic           0.149251
Aristotle          0.131354
German Idealism    0.113455
Plato              0.103314
Continental        0.090962
Phenomenology      0.076943
Rationalism        0.061798
Empiricism         0.053671
Scholasticism      0.050184
Capitalism         0.048994
Communism          0.048358
Feminism           0.028275
Stoicism           0.020342
Nietzsche          0.018349
Kierkegaard        0.004750
Name: SCHOOL, dtype: float64

In [6]:
classifier_dict = {}
for author in for_db['AUTHOR'].unique():
  classifier_dict[author] = 'AUTHOR'
for title in for_db['TITLE'].unique():
  classifier_dict[title] = 'TITLE'
for school in for_db['SCHOOL'].unique():
  classifier_dict[school] = 'SCHOOL'

In [7]:
classifier_dict

{'Plato': 'SCHOOL',
 'Aristotle': 'SCHOOL',
 'Locke': 'AUTHOR',
 'Hume': 'AUTHOR',
 'Berkeley': 'AUTHOR',
 'Spinoza': 'AUTHOR',
 'Leibniz': 'AUTHOR',
 'Descartes': 'AUTHOR',
 'Malebranche': 'AUTHOR',
 'Russell': 'AUTHOR',
 'Moore': 'AUTHOR',
 'Wittgenstein': 'AUTHOR',
 'Lewis': 'AUTHOR',
 'Quine': 'AUTHOR',
 'Popper': 'AUTHOR',
 'Kripke': 'AUTHOR',
 'Foucault': 'AUTHOR',
 'Derrida': 'AUTHOR',
 'Deleuze': 'AUTHOR',
 'Merleau-Ponty': 'AUTHOR',
 'Husserl': 'AUTHOR',
 'Heidegger': 'AUTHOR',
 'Kant': 'AUTHOR',
 'Fichte': 'AUTHOR',
 'Hegel': 'AUTHOR',
 'Marx': 'AUTHOR',
 'Lenin': 'AUTHOR',
 'Smith': 'AUTHOR',
 'Ricardo': 'AUTHOR',
 'Keynes': 'AUTHOR',
 'Epictetus': 'AUTHOR',
 'Marcus Aurelius': 'AUTHOR',
 'Nietzsche': 'SCHOOL',
 'Wollstonecraft': 'AUTHOR',
 'Beauvoir': 'AUTHOR',
 'Davis': 'AUTHOR',
 'Augustine': 'AUTHOR',
 'Anselm': 'AUTHOR',
 'Seneca': 'AUTHOR',
 'Kierkegaard': 'SCHOOL',
 'Hobbes': 'SCHOOL',
 'Plato - Complete Works': 'TITLE',
 'Aristotle - Complete Works': 'TITLE',
 'Secon

Now get an updated dropdown menu.

In [8]:
all_options = sorted([x for x in list(classifier_dict.keys())])

In [9]:
dropdown_menu = []
for source in all_options:
    dropdown_menu.append({'label': source, 'value': source})

dropdown_menu

[{'label': 'A General Theory Of Employment, Interest, And Money',
  'value': 'A General Theory Of Employment, Interest, And Money'},
 {'label': 'A Treatise Concerning The Principles Of Human Knowledge',
  'value': 'A Treatise Concerning The Principles Of Human Knowledge'},
 {'label': 'A Treatise Of Human Nature',
  'value': 'A Treatise Of Human Nature'},
 {'label': 'Analytic', 'value': 'Analytic'},
 {'label': 'Anselm', 'value': 'Anselm'},
 {'label': 'Anti-Oedipus', 'value': 'Anti-Oedipus'},
 {'label': 'Aristotle', 'value': 'Aristotle'},
 {'label': 'Aristotle - Complete Works',
  'value': 'Aristotle - Complete Works'},
 {'label': 'Augustine', 'value': 'Augustine'},
 {'label': 'Beauvoir', 'value': 'Beauvoir'},
 {'label': 'Being And Time', 'value': 'Being And Time'},
 {'label': 'Berkeley', 'value': 'Berkeley'},
 {'label': 'Beyond Good And Evil', 'value': 'Beyond Good And Evil'},
 {'label': 'Capital', 'value': 'Capital'},
 {'label': 'Capitalism', 'value': 'Capitalism'},
 {'label': 'Communi

The next cell makes pickles of each relevant slice. Not necessary since we run it all on SQL.

In [ ]:
for source in classifier_dict.keys():
        source_slice = df[df[classifier_dict[source]]==source]
        source = source.title()
        source_pkl = open(f'../search_app/slice_pickles/{source}_slice.pkl', 'wb')
        pickle.dump(source_slice, source_pkl)
        source_pkl.close()
